In [2]:
import pandas as pd
from spacy.lang.fr import French
import spacy
from line_profiler import LineProfiler
from time import time
from spacy.matcher import Matcher
#from spacy.cli.download import download
#download('fr_core_news_md')
import cProfile
import io
import pstats
import re
from collections import Counter

In [1]:
[[2019],[2017]].flatten()

AttributeError: 'list' object has no attribute 'flatten'

In [2]:
"*****A CHANGER*****"
path_to_processed = '/Users/hugomuselli/DataForGood/batch7_rse/data/processed/DPEFs/dpef_paragraphs_sentences_token_labels.pckl'

"""output_filename= "../../data/processed/DPEFs/dpef_paragraphs_sentences.csv"
df = pd.read_csv(output_filename,sep=";")
if 'tokens' in df.columns:
    df = df.drop('tokens', axis=1)"""
df = pd.read_pickle(path_to_processed)
#df.to_pickle('../../data/processed/DPEFs/dpef_paragraphs_sentences_tokens.pckl')

In [9]:
def profile(fnc):
    
    def inner(*args, **kwargs):
        pr = cProfile.Profile()
        pr.enable()
        rtval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return rtval
    return inner     
        
#@profile  
        
"""
Grammar module
"""


class Grammar(object):
    """
    Linguistic processing rules applied to sentences.
    """

    def __init__(self):
        # spaCy NLP
        self.nlp = spacy.load("fr_core_news_md")
        
    def create_match(self, matcher_name):
        matcher = Matcher(self.nlp.vocab)
        setattr(self, matcher_name, matcher)

    def add_match(self, matcher_name, pattern_name, pattern):
        getattr(self, matcher_name).add(pattern_name, None, pattern)
        
        
    def parse(self, text):
        """
        Parses text to NLP tokens.
        Args:
            text: input text
        Returns:
            tokens
        """
        tokens = None
        if text:
            # Run text through linguistic rules
            tokens = self.nlp(text)

            # Apply custom rules to token list
            #tokens = self.applyRules(tokens)
        return tokens

    def label(self, tokens):
        """
        Linguistic rules processing logic. Identifies non-informative sentences and labels them accordingly.
        Labels:
            - QUESTION: Any text ending in a question mark
            - FRAGMENT: Poorly structured sentences with limited information
        Args:
            tokens: parsed tokens
        Returns:
            label if detected else None
        """
        label = None

        if tokens:
            # Label non-informative sentences
            if self.isQuestion(tokens.text):
                label = "QUESTION"
            elif self.isFragment(tokens):
                label = "FRAGMENT"
        return label
    
    def isQuestion(self, text):
        """
        Determines if the text is a question
        Args:
            text: input text
        Returns:
            true if text is a question, false otherwise
        """
        # Questions have a ? mark at end
        return text.strip().endswith("?")
    
    """def punctuation_ok(self, doc):
        if (doc[0].shape_"""

    def isFragment(self, tokens):
        """
        Run text against linguistic rules to determine if sentence is a fragment. Fragments are non descriptive.
        Args:
            tokens: nlp document tokens
        Returns:
            true if text is a sentence fragment, false otherwise
        """
        # Nominal subject Nouns/Proper Nouns
        nouns = any([t.pos_  in ["NOUN", "PROPN"]  and t.dep_ in ["nsubj", "nsubjpass"] for t in tokens])

        # Actions (adverb, auxiliary, verb)
        action = any([t.pos_ in ["ADV", "AUX", "VERB"] for t in tokens])

        # Consider root words with a nominal subject as an action word
        action = action or any([t.dep_ in ["appos", "ROOT"] and any([x.dep_ in ["nsubj", "nsubjpass"] for x in t.children]) for t in tokens])

        # Non-punctuation tokens and multi-character words (don't count single letters which are often variables used in equations)
        words = [t.text for t in tokens if t.pos_ not in ["PUNCT", "SPACE", "SYM"] and len(t.text) > 1]

        # Valid sentences take the following form:
        #  - At least one nominal subject noun/proper noun AND
        #  - At least one action/verb AND
        #  - At least 5 words
        valid = nouns and action and len(words) >= 5

        return not valid
       
    
    def isPercent(self, tokens):
        #We should be able to work this out with entities (any([t.label_ == "PERCENT" for t in doc.ents])

        # Iterate over the tokens in the doc
        for token in tokens[:-1]:
            # Check if the token resembles a number
            if token.like_num:
                # Get the next token in the document
                next_token = tokens[token.i+1]
                # Check if the next token's text equals '%'
                if next_token.text == "%":
                    return f"{token.text}%"
                else:
                    return False
            else:
                return False
        
    def isDate(self, tokens, after=0):
        #We should be able to work this out with entities (any([t.label_ == "DATE" for t in doc.ents])

        # Iterate over the tokens in the doc
        for token in tokens:
            # Check if the token resembles a number
            if token.like_num and token.is_digit:
                if len(token)==4 and int(token.text)>after:
                    return token.text
                else:
                    return 0
    
    def commitLevel(self, tokens):
        #Match for mentioning a commitment
        self.create_match('comm_level_LOW')
        pattern1 = [{"LEMMA": {"IN": ["engager", "viser", 'objectif', 'atteindre', 'prévoir' ]}}] 
        grammar.add_match('comm_level_LOW', 'pat_LOW', pattern1)

        #Match for pattern [subj+commit_verb] such as 'Le Groupe s’engage...'
        self.create_match('comm_level_MED')
        pattern2 = [{"POS": {"IN": ["PROPN", "NOUN"]}},{"POS": "PROPN", "OP": "?"}, {"POS": "AUX", "OP": "?"},
             {"LEMMA": {"IN": ["engager", "viser"]}}]
        self.add_match('comm_level_MED', 'pat_MED', pattern2)
        level = ''
        if self.comm_level_LOW(tokens):
            level = '1'+str(self.comm_level_LOW(tokens))
            if self.comm_level_MED(tokens):
                level = '2'+str(self.comm_level_MED(tokens))
                
                # Searching for label "DATE" or "PERCENT"
                """#Should work but problem with entities
                if any([t.label_=="DATE" for t in tokens.ents]):
                    level = '3'+str([t.label_ for t in tokens.ents])"""
                if self.isPercent(tokens) or self.isDate(tokens):
                    level = '3'
        return level
    

In [40]:
#This cell can take a few minutes to run due to multiple function calls
"""grammar = Grammar()
df['tokens'] = df['sentence'].apply(grammar.parse)
df['label']=df['tokens'].apply(grammar.label)
df = df[df['label']!=('QUESTION' and 'FRAGMENT')]
df['date']=df['tokens'].apply(grammar.isDate)
df['percent']=df['tokens'].apply(grammar.isPercent)"""


In [3]:
"""On utilise ces fonction pour experimenter car sinon il faut instancier un objet Grammar a chaque modif"""

def isDate(tokens, find_context = False, earliest=0):
    #We should be able to work this out with entities (any([t.label_ == "DATE" for t in doc.ents])
    dates = []
    for token in tokens:
        
        if (token.like_num 
            and token.is_digit #token must be a digit \
            and len(token)==4  # token must be of length 4 
            and int(token.text)>earliest  #Token must be above certain date
            and token.i!=0 
            #and tokens[token.i-1].shape[0]!=X
            #and tokens[token.i-1] in before_context.values
            
            ):
            current = []
            current.append((int(token.text)))
            if find_context == True:
                if token.i!=0:
                    before = tokens[token.i-1]
                else:
                    before = ' '
                """if token.i<len(tokens)-1:
                    after.append(tokens[token.i+1])"""                   
            current.append(before)
            dates.append(current)
    result = dates if dates else False
    return result

def isNum(tokens):
    #We should be able to work this out with entities (any([t.label_ == "DATE" for t in doc.ents])
    num = []
    for token in tokens:
        if token.like_num:
            num.append(token.text)
    result = num if num else False
    return result

In [5]:
"""TODO : écrire une fonction qui vérifie en fonction du contexte que les tokens parsés sont bien des dates
Ex : si le token précedent est "année" ou "en" ou "depuis" dans la liste [Janvier, fevrier, etc...]"""

#On vérifie que date n'a pas parsé autre chose que des dates"
#Il faut également vérifier qu'on n'a pas raté de date, en vérifiant que les dates phrases ne contrenant pas de date mais un like_num ne sont pas des dates

if 'date' in df.columns:
    df.drop('date', axis=1)
df['date']=df['tokens'].apply(isDate, find_context=True)

In [11]:
#Ce set de context word permet de vérifier le contexte des dates parsées par isDate, surtout dans ceux apparaissant peu de fois
occurences_context = Counter([x[1].text for y in df[df['date']!=False]['date'].values for x in y if not (type(x[1])==str or x[1].is_punct)])
context_index = [(x[1].text,z) for y,z in zip(df[df['date']!=False]['date'].values,df[df['date']!=False]['date'].index) for x in y if not (type(x[1])==str or x[1].is_punct)]
context_words = list(occurences_context.keys())
#occurences_context.most_common()
#context_index.sort()
def print_context_sentence(word ,df=df):
    """Help verifying the sentence in which a date context word appear"""
    ind = []
    for t in context_index:
        if t[0]==word:
            ind.append(t[1])
    return df.loc[ind]
    

In [27]:
#df.iloc[52]['sentence'] #Pas à garder dans dates
#print_context_sentence('isae')['sentence'].values #Pas à garder dans date #Peut se resoudre en mettant un max pour la date
"""Dans les resultats de la commande précédente il y a beaucoup de similarités entre les phrases, à voir si on ferait une fonction qui éliminerai les quasi doublons"""
#print_context_sentence('iso')['sentence'].values #clairement pas une date #on voit un pattern emerger avec les normes
#print_context_sentence('carrefour')['sentence'].values #Il s'agit du titre d'un projet ou autre, on peut s'en affranchir avec les quote ou faire une régle "si précédé d'un mot avec majuscule..."
#print_context_sentence('by')['sentence'].values #Pareil, titre de programme
#print_context_sentence('rse')['sentence'].values #Pareil, titre d'un programme
#print_context_sentence('social')['sentence'].values #Pareil
#print_context_sentence('groupe')['sentence'].values #Cas tangeant, à reflechir
#print_context_sentence('consolider')['sentence'].values # Phrase doublon
#print_context_sentence('exercice')['sentence'].values # OK
#print_context_sentence('numéro')['sentence'].values # Non, ça indique bien des années mais ça se réfère à un numéro de loi (prend aussi 'n°')
#print_context_sentence('référence')['sentence'].values # Indique souvet "Document de référence, on garde"
#print_context_sentence('flux')['sentence'].values # Dupliqué mais ok
#print_context_sentence('rse')['sentence'].values # Toujours comme "Rapport RSE ...."
#print_context_sentence('social')['sentence'].values # Toujours comme "Souvent dans un titre"
#print_context_sentence('n')['sentence'].values #'fin' coupé en deux, doit etre amélioré
#print_context_sentence('sur')['sentence'].values #OK
#print_context_sentence('climat')['sentence'].values #OK
#print_context_sentence('natura')['sentence'].values #Non, genre de norme
#print_context_sentence('indicateur')['sentence'].values #Ok
#print_context_sentence('donnée')['sentence'].values #Ok
#print_context_sentence('tfa')['sentence'].values #Nom propre
#print_context_sentence('Chiffres')['sentence'].values #Doublon
#print_context_sentence('innovation')['sentence'].values #Nom propre
#print_context_sentence('environnementale')['sentence'].values #Nom propre
print_context_sentence('ambition')['sentence'].values #Nom propre

print_context_sentence('cible')['sentence'].values #ok

print_context_sentence('cours')['sentence'].values #ok
print_context_sentence('biodiversité')['sentence'].values #"ok

print_context_sentence('spécial')['sentence'].values #ok

print_context_sentence('thermique')['sentence'].values #nom propre

print_context_sentence('vrier')['sentence'].values #Erreur

print_context_sentence('nombre')['sentence'].values #ok


array(['Part sur le nombre 2018 total d’accords 2017'], dtype=object)

In [57]:

before_context = {'mois':['janvier', 'février', 'mars','avril', 'mai', 'juillet', 'août', 'septembre', 'octobre', 'novembre'],
                  'date':['année', 'début', 'période' ,'semestre',  'cap',' été', 'automne', 'hiver', 'printemps',  'périmètre','millésime', 'score', 'cible'
],
                 'divers':['groupe', 'référence',  'ambition', 'flux', 'objectif', 'édition', 'base','sur', 'climat', 'indicateur', 'donnée', 'exercice', 'exercices', 'échéance','ambition', 'étude', 'cours', 'biodiversité', 'lauréat', 'nombre'],
                 'time_marker':['en', 'en','depuis','fin','à','de', 'et' , 'dès', 'horizon', 'entre', 'pour' ,'avant'
],
                 }
"""pd.from_dict

data = {'col_1': [3, 2, 1, 0], 'col_2': ['a', 'b', 'c', 'd']}
pd.DataFrame.from_dict(data)"""



"pd.from_dict\n\ndata = {'col_1': [3, 2, 1, 0], 'col_2': ['a', 'b', 'c', 'd']}\npd.DataFrame.from_dict(data)"

In [51]:
before_context = {'word':['janvier', 'février', 'mars','avril', 'mai', 'juillet', 'août', 'septembre', 'octobre', 'novembre','année', 'début', 'période' ,'semestre',  'cap',' été', 'automne', 'hiver', 'printemps',  'périmètre','millésime', 'score', 'cible','groupe', 'référence',  'ambition', 'flux', 'objectif', 'édition', 'base','sur', 'climat', 'indicateur', 'donnée', 'exercice', 'exercices', 'échéance','ambition', 'étude', 'cours', 'biodiversité', 'lauréat', 'nombre','en', 'en','depuis','fin','à','de', 'et' , 'dès', 'horizon', 'entre', 'pour' ,'cap','avant']
                 }

In [50]:
#white_list_words = pd.DataFrame.from_dict(before_context)

In [52]:
def find_word(word):
    for key, value in before_context.items():
        if word in value:
            return key

In [41]:
find_word('janvier')

'mois'

In [60]:
white_list_words.to_csv('white_list_words.csv')

In [58]:
white_list_words['category']=white_list_words['word'].apply(find_word)

In [16]:
occurences_context.most_common()

[('en', 1452),
 ('En', 506),
 ('depuis', 223),
 ('fin', 151),
 ('à', 140),
 ('décembre', 134),
 ('Depuis', 114),
 ('ici', 88),
 ('année', 87),
 ('de', 81),
 ('et', 78),
 ('octobre', 65),
 ('juin', 65),
 ('mars', 62),
 ('janvier', 56),
 ('novembre', 54),
 ('septembre', 51),
 ('dès', 51),
 ('juillet', 45),
 ('horizon', 42),
 ('entre', 39),
 ('consolidé', 38),
 ('exercice', 36),
 ('mai', 36),
 ('février', 31),
 ('avril', 27),
 ('pour', 27),
 ('n°', 23),
 ('période', 23),
 ('début', 19),
 ('Fin', 19),
 ('août', 18),
 ('semestre', 16),
 ('années', 12),
 ('été', 10),
 ('trimestre', 10),
 ('Dès', 10),
 ('CAP', 9),
 ('ISAE', 8),
 ('OCTOBRE', 7),
 ('Carrefour', 7),
 ('by', 6),
 ('automne', 6),
 ('ISO', 6),
 ('flux', 6),
 ('Chiffres', 6),
 ('RSE', 6),
 ('Ambitions', 5),
 ('printemps', 5),
 ('référence', 5),
 ('Début', 5),
 ('Référence', 4),
 ('Groupe', 4),
 ('édition', 4),
 ('courant', 4),
 ('base', 4),
 ('n', 4),
 ('cembre', 4),
 ('avant', 4),
 ('ﬁn', 4),
 ('2017-', 4),
 ('résultats', 3),
 ('ob

In [313]:
context_index

[('social', 2),
 ('horizon', 13),
 ('en', 13),
 ('juillet', 21),
 ('mars', 21),
 ('août', 21),
 ('numéro', 21),
 ('référence', 27),
 ('agenda', 34),
 ('ambition', 52),
 ('en', 76),
 ('rapport', 77),
 ('août', 82),
 ('exercice', 83),
 ('en', 106),
 ('groupe', 111),
 ('exercice', 117),
 ('en', 122),
 ('et', 122),
 ('janvier', 123),
 ('exercice', 124),
 ('en', 127),
 ('août', 127),
 ('groupe', 137),
 ('depuis', 138),
 ('septembre', 140),
 ('année', 147),
 ('isae', 148),
 ('décembre', 164),
 ('janvier', 165),
 ('en', 172),
 ('en', 180),
 ('en', 182),
 ('à', 188),
 ('depuis', 204),
 ('depuis', 205),
 ('innovation', 208),
 ('ici', 213),
 ('à', 213),
 ('à', 217),
 ('en', 220),
 ('horizon', 227),
 ('en', 228),
 ('en', 230),
 ('ambition', 233),
 ('en', 251),
 ('en', 254),
 ('en', 273),
 ('en', 273),
 ('en', 280),
 ('en', 281),
 ('en', 286),
 ('en', 307),
 ('en', 307),
 ('ambition', 317),
 ('octobre', 325),
 ('depuis', 331),
 ('entre', 339),
 ('et', 339),
 ('en', 339),
 ('en', 341),
 ('octobre',

In [ ]:
"""TODO : on peut donc utiliser le fait qu'une phrase soit un fragement si elle n'a pas de quote fermante (voir s'il y a le meme attribut aved des parenthese, mais je ne pense pas)
Il y a mieux, is_right_punct et is_left_punct"""

In [4]:
"""Exemples de tout ce que Spacy n'arrive pas a faire"""

In [189]:
"""Spacy ne genère pas correctement les Named Entities """
#Le caractère « n'est pas considéré comme une ponctuation, peut etre parce qu'il n'y a pas de 
#Parenthèse fermante ou bien parce que c'est une particularité du Francais. Cherché dans les fichiers de config
#Pas trouvé comment modifier ça 
#Par contre l'atrribut .is_punct et is_quote sont TRUE... bien que .pos_ donne "PRON"

#Ici on a essayé de voir pourquoi certaines entity etaient reconnues et d'autres non, mais peut etre
#Qu'il faudra comparer ça avec des resultats de filtres

df['ents']=df['tokens'].apply(lambda x: x.ents)
df[df['ents']==()].tokens.values;

In [ ]:
"""Utile"""
dir(spacy.tokens.token.Token);

In [ ]:
"""A Faire remonter"""
"""Il y a beaucoup de quasi doublons (sort_values par sentence pour s'en rendre compte)
De nombreuses phrases sont légèrement broken (un espace en trop quelque par, manque une parenthese fermante à la fin etc...)
Presque toutes les phrase contenant un tiret son coupées en deux"""